In [1]:
import os
from glob import glob
import matplotlib.pyplot as plt
from matplotlib import patches as mpatches, colors
from matplotlib.colors import ListedColormap
import seaborn as sns
import numpy as np
import numpy.ma as ma
import pandas as pd
import rasterio as rio
from rasterio.plot import plotting_extent
from rasterio.mask import mask
import geopandas as gpd
from shapely.geometry import mapping
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import earthpy.mask as em
from shutil import copyfile

In [6]:
def change_string(s):
    s += '1'
    return s
    
s = 'abc'

print(change_string(s))

print(s)

abc1
abc


In [4]:
src_dir = 'D:/canopy_data/pipeline-products/raw_products/SAFE_ordered_cloud_cover/'

glob(src_dir + "*/*TCI_10m.jp2",recursive=True)[0]

'D:/canopy_data/pipeline-products/raw_products/SAFE_ordered_cloud_cover\\1\\T33NTB_20191110T092119_TCI_10m.jp2'

In [8]:
mask_dir = 'D:/canopy_data/pipeline-products/raw_products/SCL/'

glob(mask_dir + "*.jp2", recursive=True)[0]

'D:/canopy_data/pipeline-products/raw_products/SCL\\33NTB_20191110_SCL.jp2'

In [10]:
mask_dir = 'D:/canopy_data/pipeline-products/raw_products/MSK/'

glob(mask_dir + "*.jp2", recursive=True)[0]

'D:/canopy_data/pipeline-products/raw_products/MSK\\L2A_T33NTB_A013990_20191110T093408_MSK_CLDPRB_20m.jp2'

In [9]:
def make_tci_string(TCI_file):
    return TCI_file.split("\\")[-1].split("_")[0] + "_" + TCI_file.split("\\")[-1].split("_")[1][:8]

TCI_string = make_tci_string(glob(src_dir + "*/*TCI_10m.jp2",recursive=True)[0])

TCI_string

'T33NTB_20191110'

In [11]:
mask_dir.split('/')

['D:', 'canopy_data', 'pipeline-products', 'raw_products', 'MSK', '']

In [5]:
def make_string_match(mask, mask_type):
    if mask_type == 'MSK':
        return mask.split("\\")[-1].split("_")[1] + "_" + mask.split("\\")[-1].split("_")[3][:8]
    elif mask_type == 'SCL':
        return 'T' + mask.split("\\")[-1].split("_")[0] + "_" + mask.split("\\")[-1].split("_")[1]
    else:
        raise ValueError('Mask type must be either MSK or SCL')


def cloud_fill(source_dir, mask_dir, output_dir, start=0):
    if source_dir[-1] != '/':
        source_dir += '/'
    if mask_dir[-1] != '/':
        mask_dir += '/'
    if output_dir[-1] != '/':
        output_dir += '/'
    
    TCI_list = glob(source_dir + "*/*TCI_10m.jp2", recursive=True)
    Mask_list = glob(mask_dir + "*.jp2", recursive=True)[start:]
    mask_type = mask_dir.split('/')[-2]
    nodatavalue = int(0)
    TCI_string_match = {}

    for index,TCI_file in enumerate(TCI_list):
        TCI_string = TCI_file.split("\\")[-1].split("_")[0] + "_" + TCI_file.split("\\")[-1].split("_")[1][:8]
        TCI_string_match.update({TCI_string:index})
        
    num_masks = len(Mask_list)
    for i, mask in enumerate(Mask_list, 1):
        print(f'Mask {i} of {num_masks}')
        string_match = make_string_match(mask, mask_type)
        if string_match in TCI_string_match.keys():
            index = TCI_string_match[string_match]
            TCI_file = TCI_list[index]
            Mask_file = mask
    
            Mask_filename = Mask_file.split("\\")[-1]
            TCI_filename = "processed_" + TCI_file.split("\\")[-1]
        

            print(f"Applying {Mask_filename} to {TCI_filename}")

            with rio.open(TCI_file) as sen_TCI_src:
                sen_TCI = sen_TCI_src.read(masked=True)
                sen_TCI_meta = sen_TCI_src.meta

            with rio.open(Mask_file) as sen_mask_src:
                sen_mask_pre = sen_mask_src.read(1)
                sen_mask = np.repeat(np.repeat(sen_mask_pre,2,axis=0),2,axis=1)

            # All pixels above 0 probability will be classified as True
            if mask_type == 'MSK':
                sen_mask_qa = sen_mask > 0
            elif mask_type == 'SCL':
                sen_mask_qa = sen_mask > 6
            else:
                raise ValueError('Mask type must be either MSK or SCL')


            # Apply mask to source TCI file
            if np.count_nonzero(sen_mask_qa) > 0:
                sen_TCI_cl_free_nan = em.mask_pixels(sen_TCI, sen_mask_qa)
                sen_TCI_cl_free_processed = np.ma.filled(sen_TCI_cl_free_nan, fill_value=nodatavalue)
            else:
                sen_TCI_c1_free_processed = sen_mask_qa
        
            
            folder = TCI_file.split("\\")[-2]
            outpath = output_dir + folder + "/" + TCI_filename
    
            # Check if directory exists
            if not os.path.isdir(output_dir + folder):
                os.mkdir(output_dir + folder)


            # Export cloud-masked TCI file
            with rio.open(outpath, 'w', **sen_TCI_meta) as outf:
                outf.write(sen_TCI_cl_free_processed)
                
        else:
            print(f'Could not find match for {mask_filename}')

In [6]:
source_dir = 'D:/canopy_data/pipeline-products/raw_products/SAFE_ordered_cloud_cover/'

mask_dir = 'D:/canopy_data/pipeline-products/raw_products/MSK/'

output_dir = 'D:/canopy_data/pipeline-products/cloudless_products/MSK_cloudcover_order/'

cloud_fill(source_dir, mask_dir, output_dir)

Mask 1 of 210
Applying L2A_T33NTB_A013990_20191110T093408_MSK_CLDPRB_20m.jp2 to processed_T33NTB_20191110T092119_TCI_10m.jp2
Mask 2 of 210
Applying L2A_T33NTB_A017851_20200806T093251_MSK_CLDPRB_20m.jp2 to processed_T33NTB_20200806T092029_TCI_10m.jp2
Mask 3 of 210
Applying L2A_T33NTC_A023971_20200124T093439_MSK_CLDPRB_20m.jp2 to processed_T33NTC_20200124T092251_TCI_10m.jp2
Mask 4 of 210
Applying L2A_T33NUA_A010801_20190401T092807_MSK_CLDPRB_20m.jp2 to processed_T33NUA_20190401T090559_TCI_10m.jp2
Mask 5 of 210
Applying L2A_T33NUA_A015091_20200126T091912_MSK_CLDPRB_20m.jp2 to processed_T33NUA_20200126T091139_TCI_10m.jp2
Mask 6 of 210
Applying L2A_T33NUA_A015377_20200215T092112_MSK_CLDPRB_20m.jp2 to processed_T33NUA_20200215T091019_TCI_10m.jp2
Mask 7 of 210
Applying L2A_T33NUA_A022498_20191013T091824_MSK_CLDPRB_20m.jp2 to processed_T33NUA_20191013T090921_TCI_10m.jp2
Mask 8 of 210
Applying L2A_T33NUA_A023785_20200111T092435_MSK_CLDPRB_20m.jp2 to processed_T33NUA_20200111T091341_TCI_10m.jp2


Mask 67 of 210
Applying L2A_T33NVD_A019824_20190409T093304_MSK_CLDPRB_20m.jp2 to processed_T33NVD_20190409T092031_TCI_10m.jp2
Mask 68 of 210
Applying L2A_T33NVD_A023828_20200114T093418_MSK_CLDPRB_20m.jp2 to processed_T33NVD_20200114T092331_TCI_10m.jp2
Mask 69 of 210
Applying L2A_T33NVD_A024257_20200213T093321_MSK_CLDPRB_20m.jp2 to processed_T33NVD_20200213T092101_TCI_10m.jp2
Mask 70 of 210
Applying L2A_T33NVD_A024686_20200314T092857_MSK_CLDPRB_20m.jp2 to processed_T33NVD_20200314T092031_TCI_10m.jp2
Mask 71 of 210
Applying L2A_T33NWA_A015377_20200215T092112_MSK_CLDPRB_20m.jp2 to processed_T33NWA_20200215T091019_TCI_10m.jp2
Mask 72 of 210
Applying L2A_T33NWA_A016378_20200425T092712_MSK_CLDPRB_20m.jp2 to processed_T33NWA_20200425T090549_TCI_10m.jp2
Mask 73 of 210
Applying L2A_T33NWA_A017665_20200724T092637_MSK_CLDPRB_20m.jp2 to processed_T33NWA_20200724T090559_TCI_10m.jp2
Mask 74 of 210
Applying L2A_T33NWA_A023785_20200111T092435_MSK_CLDPRB_20m.jp2 to processed_T33NWA_20200111T091341_TCI_

Mask 132 of 210
Applying L2A_T34MBB_A017436_20200708T090330_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200708T084559_TCI_10m.jp2
Mask 133 of 210
Applying L2A_T34MBB_A017722_20200728T085918_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200728T084559_TCI_10m.jp2
Mask 134 of 210
Applying L2A_T34MBB_A017865_20200807T090515_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200807T084559_TCI_10m.jp2
Mask 135 of 210
Applying L2A_T34MBB_A025415_20200504T090346_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200504T084601_TCI_10m.jp2
Mask 136 of 210
Applying L2A_T34MBB_A025701_20200524T090953_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200524T084601_TCI_10m.jp2
Mask 137 of 210
Applying L2A_T34MBB_A025987_20200613T090356_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200613T084611_TCI_10m.jp2
Mask 138 of 210
Applying L2A_T34MBB_A026273_20200703T090351_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200703T084601_TCI_10m.jp2
Mask 139 of 210
Applying L2A_T34MBB_A026559_20200723T090956_MSK_CLDPRB_20m.jp2 to processed_T34MBB_20200723T084

Mask 197 of 210
Applying L2A_T34NFG_A016535_20200506T085121_MSK_CLDPRB_20m.jp2 to processed_T34NFG_20200506T083559_TCI_10m.jp2
Mask 198 of 210
Applying L2A_T34NFG_A023513_20191223T085353_MSK_CLDPRB_20m.jp2 to processed_T34NFG_20191223T084351_TCI_10m.jp2
Mask 199 of 210
Applying L2A_T34NFG_A024228_20200211T085547_MSK_CLDPRB_20m.jp2 to processed_T34NFG_20200211T084101_TCI_10m.jp2
Mask 200 of 210
Applying L2A_T34NFG_A024514_20200302T085853_MSK_CLDPRB_20m.jp2 to processed_T34NFG_20200302T083841_TCI_10m.jp2
Mask 201 of 210
Applying L2A_T34NFH_A013961_20191108T084733_MSK_CLDPRB_20m.jp2 to processed_T34NFH_20191108T084059_TCI_10m.jp2
Mask 202 of 210
Applying L2A_T34NFH_A014390_20191208T085450_MSK_CLDPRB_20m.jp2 to processed_T34NFH_20191208T084239_TCI_10m.jp2
Mask 203 of 210
Applying L2A_T34NFH_A014533_20191218T084754_MSK_CLDPRB_20m.jp2 to processed_T34NFH_20191218T084249_TCI_10m.jp2
Mask 204 of 210
Applying L2A_T34NFH_A014676_20191228T085358_MSK_CLDPRB_20m.jp2 to processed_T34NFH_20191228T084